In [2]:
import pandas as pd
import numpy as np

# Generate synthetic data
def generate_data(num_samples=1000):
    np.random.seed(42)
    data = {
        'connection_duration': np.random.normal(10, 5, num_samples),
        'signal_strength': np.random.normal(-50, 10, num_samples),
        'data_transfer_rate': np.random.normal(5, 2, num_samples),
        'time_of_day': np.random.randint(0, 24, num_samples),
        'intrusion': np.random.randint(0, 2, num_samples)
    }
    return pd.DataFrame(data)

# Save to CSV
df = generate_data()
df.to_csv('../data/bluetooth_data.csv', index=False)